In [1]:
!pip install openai pinecone-client datasets


Defaulting to user installation because normal site-packages is not writeable


In [2]:
pip install 'pinecone-client[grpc]'


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import openai

from typing import List, Iterator
import pandas as pd
import numpy as np
import os
from ast import literal_eval

# Pinecone's client library for Python
import pinecone

# Ignore unclosed SSL socket warnings - optional in case you get these errors
import warnings

warnings.filterwarnings(action="ignore", message="unclosed", category=ResourceWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

/Users/akshitapathania/Library/Python/3.9/lib/python/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
article_df = pd.read_csv('/Users/akshitapathania/Desktop/Assign5_CSV.csv')
article_df.head()

,Title,Year,Level,Introduction Summary,Learning Outcomes,Link to Summary Page,Link to PDF File
0,Industry and Company Analysis (2018),2018\n Curriculum,Level II,This reading explores industry and company ana...,"compare top-down, bottom-up, and hybrid approa...",https://www.cfainstitute.org/membership/profes...,No PDF link found
1,Free Cash Flow Valuation (2018),2018\n Curriculum,Level II,Discounted cash flow (DCF) valuation views the...,compare the free cash flow to the firm (FCFF) ...,https://www.cfainstitute.org/membership/profes...,No PDF link found
2,Residual Income Valuation (2018),2018\n Curriculum,Level II,Residual income models of equity value have be...,"calculate and interpret residual income, econo...",https://www.cfainstitute.org/membership/profes...,No PDF link found
3,Market-Based Valuation: Price and Enterprise V...,2018\n Curriculum,Level II,Among the most familiar and widely used valuat...,distinguish between the method of comparables ...,https://www.cfainstitute.org/membership/profes...,No PDF link found


In [6]:
data = article_df['Introduction Summary']

In [7]:
data

0    This reading explores industry and company ana...
1    Discounted cash flow (DCF) valuation views the...
2    Residual income models of equity value have be...
3    Among the most familiar and widely used valuat...
Name: Introduction Summary, dtype: object

In [8]:
import tiktoken

tokenizer = tiktoken.get_encoding('p50k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

In [10]:
from uuid import uuid4
from tqdm.auto import tqdm

# Assuming 'Learning Outcomes' is the name of the column in the DataFrame 'article_df_s'
chunks = []

for idx, text in tqdm(article_df['Introduction Summary'].items(), total=article_df['Introduction Summary'].shape[0]):
    # Assuming text_splitter is defined and has a method split_text that splits the text into chunks
    texts = text_splitter.split_text(text)
    chunks.extend([{
        'id': str(uuid4()),
        'text': chunk_text,
        'chunk': i,
        'title': article_df.at[idx, 'Title']  
    } for i, chunk_text in enumerate(texts)])



100%|██████████| 4/4 [00:00<00:00, 360.59it/s]


## Initialise the embedded model

In [11]:
# initialize openai API key
from openai import OpenAI
api_key = os.getenv("api_key")


embed_model = "text-embedding-3-small"

client = OpenAI(api_key=api_key)
res = client.embeddings.create(
    input=[
        "Sample document text goes here",
        "there will be several phrases in each batch"
    ], model = embed_model
)

In [12]:
print(res)

CreateEmbeddingResponse(data=[Embedding(embedding=[-0.0030704932287335396, 0.011769662611186504, -0.00524107227101922, -0.027138981968164444, -0.016434386372566223, 0.03246431425213814, -0.01631304807960987, -0.0009993427665904164, -0.025952577590942383, -0.006484773941338062, 0.020101450383663177, 0.016636613756418228, -0.009160922840237617, 0.023458434268832207, -0.010199026204645634, 0.013488599099218845, 0.025251520797610283, -0.016852322965860367, 0.012167377397418022, -0.016245638951659203, -0.004273748956620693, -0.0064780330285429955, -0.0044186790473759174, 0.020748579874634743, -0.010549554601311684, -0.0037445861380547285, 0.013704309239983559, -0.02627614140510559, -0.0004048770642839372, -0.002160467905923724, 0.0058713494800031185, -0.010077688843011856, -0.028177084401249886, -0.016151266172528267, -0.004250155761837959, 0.007475690450519323, -0.0028564687818288803, -0.03154755011200905, 0.024011189118027687, -0.03346197307109833, -0.00034968569525517523, 0.0130774024873

## Initialising the Index

In [13]:
from pinecone import Pinecone, PodSpec

papi_key2 = os.getenv("papi_key2")
pinecone = Pinecone(api_key=papi_key2)

index_name = 'qa-generation'  # Valid index name with lower case and hyphen

# Create the index
pod_spec = PodSpec(environment="gcp-starter")
pinecone.create_index(name=index_name, dimension=len(res.data[0].embedding), spec=pod_spec)

# Initialize the index
index = pinecone.Index(name=index_name)

# Confirm our index was created
print(pinecone.list_indexes())


{'indexes': [{'dimension': 1536,
              'host': 'qa-generation-9vrqwzu.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'qa-generation',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}


In [14]:
from tqdm.auto import tqdm
import datetime
from time import sleep

batch_size = 100  # how many embeddings we create and insert at once

# Function to create embeddings with retry logic
def create_embeddings_with_retry(texts, model, max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            return client.embeddings.create(input=texts, model=model)
        except Exception as e:
            print(f"Attempt {retries + 1}: Encountered an error - {e}")
            sleep(5)  # wait before retrying
            retries += 1
    raise Exception(f"Failed to create embeddings after {max_retries} retries.")

# Process the data in batches
for i in tqdm(range(0, len(article_df), batch_size)):
    i_end = min(len(article_df), i + batch_size)
    batch = article_df.iloc[i:i_end]

    ids_batch = [str(uuid4()) for _ in range(len(batch))]
    texts = batch['Introduction Summary'].tolist()

    res = create_embeddings_with_retry(texts, embed_model)

    embeds = [record.embedding for record in res.data]

    # Prepare data for Pinecone upsert
    for idx, (text, embed) in enumerate(zip(texts, embeds)):
        # Use a relevant column or derived attribute to determine the namespace
        title = batch.iloc[idx]['Title']  
        
        # Clean the title to ensure it only contains ASCII-printable characters
        title_cleaned = ''.join(char for char in title if char.isascii())
        namespace = f"{title_cleaned}_Summary"  # Create a namespace based on the cleaned title
        namespace = ''.join(char for char in namespace if char.isalnum())  # Remove non-alphanumeric characters
        
        to_upsert = [(ids_batch[idx], embed, {'text': text, 'chunk': idx, 'url': 'default_url'})]
        index.upsert(vectors=to_upsert, namespace=namespace)


100%|██████████| 1/1 [00:01<00:00,  1.45s/it]


In [15]:
from openai import OpenAI


# Initialize OpenAI API key
api_key = os.getenv("api_key")

query = " Generate 50 questions based on the Introduction Summary "
embed_model = "text-embedding-3-small" 

# Generate the embedding for the query
client = OpenAI(api_key=api_key)
res = client.embeddings.create(
    model=embed_model,
    input=query
)

# Retrieve the embedding from the response
embedding_vector = res.data[0].embedding



## Building a knowledge base

In [16]:
embed_model = "text-embedding-3-small"

client = OpenAI(api_key=api_key)
res = client.embeddings.create(
    input=[
        "https://www.cfainstitute.org/-/media/documents/support/programs/cfa/sample-level-II-itemset-questions.pdf",
        "https://www.cfainstitute.org/-/media/documents/support/programs/cfa/sample-level-I-questions.pdf",
        "https://www.cfainstitute.org/-/media/documents/support/programs/cfa/sample-level-III-itemset-questions.pdf"
    ], model= embed_model
)

In [18]:
from pinecone import Pinecone
papi_key2 = os.getenv("papi_key2")
pinecone = Pinecone(api_key=papi_key2)

## Generating 50 Questions and answers for the Free Cash Flow Valuation 2018 (SetA)

In [20]:
from openai import OpenAI

# Initialize OpenAI API key
api_key = os.getenv("api_key")

# Adjust the query to indicate the need for 50 questions with answers
query = "Generate questions and answers with 4 options each, with the correct option being bold, based on the Introduction Summary. Provide 50 questions along with their answers."

embed_model = "text-embedding-3-small"  

# Generate the embedding for the query
client = OpenAI(api_key=api_key)
res = client.embeddings.create(
    model=embed_model,
    input=query
)

# Retrieve the embedding from the response
embedding_vector = res.data[0].embedding

from IPython.display import Markdown

primer = """You are a Q&A bot, designed to provide intelligent answers based on the information provided. If the information is not available, you truthfully say, "I don't know"."""

# Initialize the list of augmented queries with answers
augmented_queries_FreeCashFlowValuation2018Summary = []

# Increase the top_k parameter to retrieve more documents
query_result_FreeCashFlowValuation2018Summary = index.query(vector=embedding_vector, top_k=20, namespace="FreeCashFlowValuation2018Summary", include_metadata=True)

# Get list of retrieved text from the query result
contexts_FreeCashFlowValuation2018Summary = [item['metadata']['text'] for item in query_result_FreeCashFlowValuation2018Summary['matches']]

# Loop through each context and generate questions with answers
total_questions = 0
for context in contexts_FreeCashFlowValuation2018Summary:
    # Check if we have already generated 50 questions
    if total_questions >= 50:
        break

    for _ in range(2):  # Run the loop twice for each context
        # Check if we have already generated 50 questions
        if total_questions >= 50:
            break

        # Update the message to request questions with answers
        messages = [{"role": "system", "content": primer}, {"role": "user", "content": context + " Generate questions with 4 options and underline the correct one."}]
        res = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=1500,  
            n=25  # Number of completions (questions with answers) to generate per iteration
        )

        # Append the generated questions and answers to the list
        for choice in res.choices:
            augmented_queries_FreeCashFlowValuation2018Summary.append(choice.message.content)
            total_questions += 1
            if total_questions >= 50:
                break


# Display the augmented queries with answers
for idx, query in enumerate(augmented_queries_FreeCashFlowValuation2018Summary):
    display(Markdown(f"Query {idx + 1} for FreeCashFlowValuation2018Summary:\n\n{query}"))
    print("=" * 50)


Query 1 for FreeCashFlowValuation2018Summary:

What is the purpose of discounted cash flow (DCF) valuation when applied to dividends?

A) To calculate the intrinsic value of a security based on past cash flows
B) To estimate the future cash flows available for distribution to shareholders
C) To value common equity directly without considering non-common-stock capital
D) To determine the historical cash flows paid to stockholders

The correct answer is _B) To estimate the future cash flows available for distribution to shareholders_.

Query 2 for FreeCashFlowValuation2018Summary:

What is one of the advantages of using free cash flow models over dividend discount models (DDMs)?  
A) Free cash flows are difficult to calculate  
B) Free cash flows provide a more economically sound basis for valuation  
C) Dividend discount models account for nonoperating assets  
D) Dividend discount models require forecasting future cash flows

**_B) Free cash flows provide a more economically sound basis for valuation_**

Query 3 for FreeCashFlowValuation2018Summary:

What is the purpose of this reading?

A) To develop the background required to use DDM for valuation
B) To explore the constant-growth models for valuing FCFF and FCFE
C) To define the concepts of free cash flow to the firm and free cash flow to equity
D) To explain the importance of forecasting dividends for valuation

The correct answer is C) To define the concepts of free cash flow to the firm and free cash flow to equity

Query 4 for FreeCashFlowValuation2018Summary:

What is the purpose of using the FCFF or FCFE approaches to value a company’s equity?

A) To complicate the valuation process
B) To simplify the valuation process
C) To confuse analysts with complex financial information
D) To increase the number of steps required in valuation

Correct answer: B) To simplify the valuation process

Query 5 for FreeCashFlowValuation2018Summary:

What is the main focus of discounted cash flow (DCF) valuation?
a) Analyzing historical stock prices
b) Valuing a security based on past performance
c) Present value of expected future cash flows
d) Predicting short-term market trends

_**c) Present value of expected future cash flows**_

...

Query 46 for FreeCashFlowValuation2018Summary:

What is the main focus of Discounted Cash Flow (DCF) valuation?
1. Present value of current market price
2. Present value of expected future cash flows
3. Present value of historical cash flows
4. Present value of book value

The correct answer is: _Present value of expected future cash flows_

Query 47 for FreeCashFlowValuation2018Summary:

What is the purpose of using the FCFF or FCFE approaches to value a company’s equity?

A) To complicate the valuation process
B) To make forecasting future cash flows easier
C) To provide an economically sound basis for valuation
D) To rely solely on dividend discount models

Correct answer: C) To provide an economically sound basis for valuation

Query 48 for FreeCashFlowValuation2018Summary:

What are some key considerations when using free cash flow models to value a company's equity?

A) Free cash flows provide an unsound basis for valuation.\
B) Dividends are a more reliable measure of a company's value compared to free cash flows.\
C) FCFF and FCFE are readily available data.\
D) Common equity can be valued directly using FCFE or indirectly by first using a FCFF model to estimate the value of the firm and then subtracting the value of non-common-stock capital from FCFF to arrive at an estimate of the value of equity.

The correct answer is _D) Common equity can be valued directly using FCFE or indirectly by first using a FCFF model to estimate the value of the firm and then subtracting the value of non-common-stock capital from FCFF to arrive at an estimate of the value of equity_.

Query 49 for FreeCashFlowValuation2018Summary:

What are some considerations when applying Free Cash Flow (FCF) valuation models to value a company's equity securities?

A) FCFF and FCFE are readily available data and do not require calculations\
B) DCF analysis is not applicable when using FCFF and FCFE\
C) Analysts do not need to understand a company's financial statements or industry when using FCFF and FCFE\
D) The value of common equity can be estimated using FCFF by subtracting non-common stock capital

**_Correct answer:_** D) The value of common equity can be estimated using FCFF by subtracting non-common stock capital

Query 50 for FreeCashFlowValuation2018Summary:

Which type of valuation model is extended in the reading to value a company and its equity securities?  
A) Price-to-earnings ratio approach  
B) Dividend growth model  
C) Discounted cash flow analysis  
D) Relative valuation methods  

**_Correct answer: C) Discounted cash flow analysis_**

## Storing the 50 Questions and answers for the Free Cash Flow Valuation 2018 (SetA)

In [21]:
from openai import OpenAI

# Initialize OpenAI API key
api_key = os.getenv("api_key")
client = OpenAI(api_key=api_key)

import uuid

# Assuming 'augmented_queries_FreeCashFlowValuation2018Summary' is already a list of tuples (question, answer)
augmented_queries = augmented_queries_FreeCashFlowValuation2018Summary  # Directly use the list

# Function to create embeddings with retry logic
def create_embeddings_with_retry(texts, model, max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            # Ensure that 'input' is a list of strings
            if not isinstance(texts, list):
                texts = [texts]
            return client.embeddings.create(input=texts, model=model)
        except Exception as e:
            print(f"Attempt {retries + 1}: Encountered an error - {e}")
            sleep(5)  # wait before retrying
            retries += 1
    raise Exception(f"Failed to create embeddings after {max_retries} retries.")

# Extract questions and answers for embedding
texts_to_embed = [qa[0] + " " + qa[1] for qa in augmented_queries]  # Combine questions and answers for embedding

# Generate embeddings for the questions and answers
embed_model = "text-embedding-3-small"
qa_embeddings = create_embeddings_with_retry(texts_to_embed, embed_model)

# Prepare data for Pinecone upsert
upsert_data = []
for idx, qa_combined in enumerate(augmented_queries):
    if not isinstance(qa_combined, str):
        print(f"Skipping item at index {idx} because it is not a string.")
        continue

    # Assuming you have a way to split qa_combined into question and answer
    parts = qa_combined.split('?', 1)
    question = parts[0].strip() + '?'
    answer = parts[1].strip() if len(parts) > 1 else ""

    # Proceed to create embeddings and upsert to Pinecone as before
    # Generate a unique ID for each query-answer pair
    qa_id = str(uuid.uuid4())
    
    # Assuming you have already generated embeddings for all QA pairs
    embed = qa_embeddings.data[idx].embedding
    
    # Prepare the upsert data with metadata
    upsert_data.append((qa_id, embed, {'question': question, 'answer': answer}))

# Perform the upsert operation to Pinecone
namespace = "SetA_FreeCashFlowValuation2018Summary"  
index.upsert(vectors=upsert_data, namespace=namespace)

print(f"Upserted {len(upsert_data)} questions and answers into Pinecone index under namespace '{namespace}'.")



Upserted 50 questions and answers into Pinecone index under namespace 'SetA_FreeCashFlowValuation2018Summary'.


## Generating 50 Questions for SetB for Free Cash Flow Valuation 2018 (SetB)

In [38]:
from openai import OpenAI

# Initialize OpenAI API key
api_key = os.getenv("api_key")

# Adjust the query to indicate the need for 50 questions with answers
query = "Generate 50 questions and answers with 4 options each, with the correct option in the format of 'The Correct Answer is:' followed by the correct option. Make sure all the questions have answers and they are in the same format."

embed_model = "text-embedding-3-small"  

# Generate the embedding for the query
client = OpenAI(api_key=api_key)
res = client.embeddings.create(
    model=embed_model,
    input=query
)

# Retrieve the embedding from the response
embedding_vector = res.data[0].embedding

from IPython.display import Markdown

primer = """You are a Q&A bot, designed to provide intelligent answers based on the information provided. If the information is not available, you truthfully say, "I don't know"."""

# Initialize the list of augmented queries with answers
augmented_queries_FreeCashFlowValuation2018Summary = []

# Increase the top_k parameter to retrieve more documents
query_result_FreeCashFlowValuation2018Summary = index.query(vector=embedding_vector, top_k=20, namespace="FreeCashFlowValuation2018Summary", include_metadata=True)

# Get list of retrieved text from the query result
contexts_FreeCashFlowValuation2018Summary = [item['metadata']['text'] for item in query_result_FreeCashFlowValuation2018Summary['matches']]

# Loop through each context and generate questions with answers
total_questions = 0
for context in contexts_FreeCashFlowValuation2018Summary:
    # Check if we have already generated 50 questions
    if total_questions >= 50:
        break

    for _ in range(2):  # Run the loop twice for each context
        # Check if we have already generated 50 questions
        if total_questions >= 50:
            break

        # Update the message to request questions with answers
        messages = [{"role": "system", "content": primer}, {"role": "user", "content": context + " Generate questions with 4 options and underline the correct one."}]
        res = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=1500,  
            n=25  # Number of completions (questions with answers) to generate per iteration
        )

        # Append the generated questions and answers to the list
        for choice in res.choices:
            augmented_queries_FreeCashFlowValuation2018Summary.append(choice.message.content)
            total_questions += 1
            if total_questions >= 50:
                break

# File path for the output file
file_path = "SetB_FreeCashFlowValuation2018Summary_Questions.txt"

# Save the augmented queries with answers to a text file
with open(file_path, 'w', encoding='utf-8') as file:
    for idx, query in enumerate(augmented_queries_FreeCashFlowValuation2018Summary):
        file_content = f"Query {idx + 1} for FreeCashFlowValuation2018Summary:\n\n{query}\n"
        file.write(file_content)
        file.write("=" * 50 + "\n")

print(f"Output saved to {file_path}")

Output saved to SetB_FreeCashFlowValuation2018Summary_Questions.txt


## Generating 50 Questions and answer pairs for Industry and Company Analysis 2018 (SetA)

In [23]:
from openai import OpenAI

# Initialize OpenAI API key
api_key = os.getenv("api_key")

# Adjust the query to indicate the need for 50 questions with answers
query = "Generate questions and answers with 4 options each, with the correct option being bold, based on the Introduction Summary. Provide 50 questions along with their answers."

embed_model = "text-embedding-3-small"  

# Generate the embedding for the query
client = OpenAI(api_key=api_key)
res = client.embeddings.create(
    model=embed_model,
    input=query
)

# Retrieve the embedding from the response
embedding_vector = res.data[0].embedding

from IPython.display import Markdown

primer = """You are a Q&A bot, designed to provide intelligent answers based on the information provided. If the information is not available, you truthfully say, "I don't know"."""

# Initialize the list of augmented queries with answers
augmented_queries_IndustryandCompanyAnalysis2018Summary = []

# Increase the top_k parameter to retrieve more documents
query_result_IndustryandCompanyAnalysis2018Summary = index.query(vector=embedding_vector, top_k=20, namespace="IndustryandCompanyAnalysis2018Summary", include_metadata=True)

# Get list of retrieved text from the query result
contexts_IndustryandCompanyAnalysis2018Summary = [item['metadata']['text'] for item in query_result_IndustryandCompanyAnalysis2018Summary['matches']]

# Loop through each context and generate questions with answers
total_questions = 0
for context in contexts_IndustryandCompanyAnalysis2018Summary:
    # Check if we have already generated 50 questions
    if total_questions >= 50:
        break

    for _ in range(2):  # Run the loop twice for each context
        # Check if we have already generated 50 questions
        if total_questions >= 50:
            break

        # Update the message to request questions with answers
        messages = [{"role": "system", "content": primer}, {"role": "user", "content": context + " Generate questions with 4 options and underline the correct one."}]
        res = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=1500,  
            n=25  # Number of completions (questions with answers) to generate per iteration
        )

        # Append the generated questions and answers to the list
        for choice in res.choices:
            augmented_queries_IndustryandCompanyAnalysis2018Summary.append(choice.message.content)
            total_questions += 1
            if total_questions >= 50:
                break


# Display the augmented queries with answers
for idx, query in enumerate(augmented_queries_IndustryandCompanyAnalysis2018Summary):
    display(Markdown(f"Query {idx + 1} for IndustryandCompanyAnalysis2018Summary:\n\n{query}"))
    print("=" * 50)




Query 1 for IndustryandCompanyAnalysis2018Summary:

What is the focus of this reading material?

A) Analyzing historical financial results  
B) Forecasting a company's future financial results  
C) Reviewing industry regulations  
D) Studying macroeconomic trends

Correct Answer: B) Forecasting a company's future financial results

Query 2 for IndustryandCompanyAnalysis2018Summary:

What is the focus of industry and company analysis for financial forecasting?

a) Analyzing historical stock prices  
b) Identifying key revenue and cost drivers  
c) Reviewing social media trends  
d) Speculating on political outcomes  

**_b) Identifying key revenue and cost drivers_**

Query 3 for IndustryandCompanyAnalysis2018Summary:

What is the starting point for an analyst when developing a forecast model?

1. Analyzing historical stock prices
2. Reviewing the company and its environment
3. Studying macroeconomic trends
4. Assessing government regulations

Please select the correct option.

Query 4 for IndustryandCompanyAnalysis2018Summary:

What is a key consideration for analysts when developing a forecast model?
1. Focusing solely on historical results
2. Ignoring the external environment
3. Understanding the fundamental drivers of the business
4. Neglecting to assess future events

_Answer: 3. Understanding the fundamental drivers of the business_

Query 5 for IndustryandCompanyAnalysis2018Summary:

What is the focus of industry and company analysis for financial forecasting?
1) Historical financial results
2) Industry trends and corporate disclosures
3) Stock market performance
4) Macroeconomic factors
_(2) Industry trends and corporate disclosures_

...

Query 46 for IndustryandCompanyAnalysis2018Summary:

What is the primary focus of the reading on industry and company analysis?

A. Identifying key revenue drivers
B. Evaluating the impact of competitive factors
C. Forecasting stock prices
D. Analyzing historical financial data

Please underline the correct answer.

Query 47 for IndustryandCompanyAnalysis2018Summary:

What is the focus of this reading on industry and company analysis?  
A) Historical events analysis  
B) Forecasting future financial results  
C) Stock market analysis  
D) Political environment assessment  

**_B) Forecasting future financial results_**

Query 48 for IndustryandCompanyAnalysis2018Summary:

What is the first step an analyst takes in developing a forecast model for a company?

1. Analyzing revenue drivers only
2. Reviewing the company and its environment
3. Estimating market multiples
4. Focusing on historical results

_**Underline the correct option.**_

Query 49 for IndustryandCompanyAnalysis2018Summary:

What is the focus of this reading on industry and company analysis?
1) Forecasting weather patterns based on historical data
2) Analyzing consumer trends in the market
3) Using industry information and corporate disclosures to forecast a company’s future financial results
4) Reviewing historical stock prices to predict future market movements

Correct answer: 3) Using industry information and corporate disclosures to forecast a company’s future financial results

Query 50 for IndustryandCompanyAnalysis2018Summary:

What is the focus of financial forecasting for equity valuation based on in the reading?  
A) Market speculation  
B) Historical data only  
C) Discounted cash flows and market multiples  
D) Analyst intuition

**_C) Discounted cash flows and market multiples_**

## Storing the 50 Questions and answers pair for Industry and Company Analysis 2018 (SetA)

In [24]:
from openai import OpenAI

# Initialize OpenAI API key
api_key = os.getenv("api_key")
client = OpenAI(api_key=api_key)

import uuid

# Assuming 'augmented_queries_IndustryandCompanyAnalysis2018Summary' is already a list of tuples (question, answer)
augmented_queries = augmented_queries_IndustryandCompanyAnalysis2018Summary  # Directly use the list

# Function to create embeddings with retry logic
def create_embeddings_with_retry(texts, model, max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            # Ensure that 'input' is a list of strings
            if not isinstance(texts, list):
                texts = [texts]
            return client.embeddings.create(input=texts, model=model)
        except Exception as e:
            print(f"Attempt {retries + 1}: Encountered an error - {e}")
            sleep(5)  # wait before retrying
            retries += 1
    raise Exception(f"Failed to create embeddings after {max_retries} retries.")

# Extract questions and answers for embedding
texts_to_embed = [qa[0] + " " + qa[1] for qa in augmented_queries]  # Combine questions and answers for embedding

# Generate embeddings for the questions and answers
embed_model = "text-embedding-3-small"
qa_embeddings = create_embeddings_with_retry(texts_to_embed, embed_model)

# Prepare data for Pinecone upsert
upsert_data = []
for idx, qa_combined in enumerate(augmented_queries):
    if not isinstance(qa_combined, str):
        print(f"Skipping item at index {idx} because it is not a string.")
        continue

    # Assuming you have a way to split qa_combined into question and answer
    parts = qa_combined.split('?', 1)
    question = parts[0].strip() + '?'
    answer = parts[1].strip() if len(parts) > 1 else ""

    # Proceed to create embeddings and upsert to Pinecone as before
    # Generate a unique ID for each query-answer pair
    qa_id = str(uuid.uuid4())
    
    # Assuming you have already generated embeddings for all QA pairs
    embed = qa_embeddings.data[idx].embedding
    
    # Prepare the upsert data with metadata
    upsert_data.append((qa_id, embed, {'question': question, 'answer': answer}))

# Perform the upsert operation to Pinecone
namespace = "SetA_IndustryandCompanyAnalysis2018Summary"  
index.upsert(vectors=upsert_data, namespace=namespace)

print(f"Upserted {len(upsert_data)} questions and answers into Pinecone index under namespace '{namespace}'.")


Upserted 50 questions and answers into Pinecone index under namespace 'SetA_IndustryandCompanyAnalysis2018Summary'.


## Generating 50 Questions for Industry and Company Analysis 2018 (SetB)

In [25]:
from openai import OpenAI

# Initialize OpenAI API key
api_key = os.getenv("api_key")

# Adjust the query to indicate the need for 50 questions with answers
query = "Generate 50 questions and answers with 4 options each, with the correct option being bold, based on the Introduction Summary."

embed_model = "text-embedding-3-small"  

# Generate the embedding for the query
client = OpenAI(api_key=api_key)
res = client.embeddings.create(
    model=embed_model,
    input=query
)

# Retrieve the embedding from the response
embedding_vector = res.data[0].embedding

from IPython.display import Markdown

primer = """You are a Q&A bot, designed to provide intelligent answers based on the information provided. If the information is not available, you truthfully say, "I don't know"."""

# Initialize the list of augmented queries with answers
augmented_queries_IndustryandCompanyAnalysis2018Summary = []

# Increase the top_k parameter to retrieve more documents
query_result_IndustryandCompanyAnalysis2018Summary = index.query(vector=embedding_vector, top_k=20, namespace="IndustryandCompanyAnalysis2018Summary", include_metadata=True)

# Get list of retrieved text from the query result
contexts_IndustryandCompanyAnalysis2018Summary = [item['metadata']['text'] for item in query_result_IndustryandCompanyAnalysis2018Summary['matches']]

# Loop through each context and generate questions with answers
total_questions = 0
for context in contexts_IndustryandCompanyAnalysis2018Summary:
    # Check if we have already generated 50 questions
    if total_questions >= 50:
        break

    for _ in range(2):  # Run the loop twice for each context
        # Check if we have already generated 50 questions
        if total_questions >= 50:
            break

        # Update the message to request questions with answers
        messages = [{"role": "system", "content": primer}, {"role": "user", "content": context + " Generate questions with 4 options and underline the correct one."}]
        res = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=1500,  
            n=25  # Number of completions (questions with answers) to generate per iteration
        )

        # Append the generated questions and answers to the list
        for choice in res.choices:
            augmented_queries_IndustryandCompanyAnalysis2018Summary.append(choice.message.content)
            total_questions += 1
            if total_questions >= 50:
                break


# File path for the output file
file_path = "SetB_IndustryandCompanyAnalysis2018Summary_Questions.txt"

# Save the augmented queries with answers to a text file
with open(file_path, 'w', encoding='utf-8') as file:
    for idx, query in enumerate(augmented_queries_IndustryandCompanyAnalysis2018Summary):
        file_content = f"Query {idx + 1} for IndustryandCompanyAnalysis2018Summary:\n\n{query}\n"
        file.write(file_content)
        file.write("=" * 50 + "\n")

print(f"Output saved to {file_path}")


Output saved to SetB_IndustryandCompanyAnalysis2018Summary_Questions.txt


## Generating the 50 Question and answers for the Market Based Valuation Price and Enterprise Value Multiples 2018 (SetA)

In [26]:
from openai import OpenAI

# Initialize OpenAI API key
api_key = os.getenv("api_key")

# Adjust the query to indicate the need for 50 questions with answers
query = "Generate questions and answers with 4 options each, with the correct option being bold, based on the Introduction Summary. Provide 50 questions along with their answers."

embed_model = "text-embedding-3-small"  

# Generate the embedding for the query
client = OpenAI(api_key=api_key)
res = client.embeddings.create(
    model=embed_model,
    input=query
)

# Retrieve the embedding from the response
embedding_vector = res.data[0].embedding

from IPython.display import Markdown

primer = """You are a Q&A bot, designed to provide intelligent answers based on the information provided. If the information is not available, you truthfully say, "I don't know"."""

# Initialize the list of augmented queries with answers
augmented_queries_MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary = []

# Increase the top_k parameter to retrieve more documents
query_result_MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary = index.query(vector=embedding_vector, top_k=20, namespace="MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary", include_metadata=True)

# Get list of retrieved text from the query result
contexts_MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary = [item['metadata']['text'] for item in query_result_MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary['matches']]

# Loop through each context and generate questions with answers
total_questions = 0
for context in contexts_MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary:
    # Check if we have already generated 50 questions
    if total_questions >= 50:
        break

    for _ in range(2):  # Run the loop twice for each context
        # Check if we have already generated 50 questions
        if total_questions >= 50:
            break

        # Update the message to request questions with answers
        messages = [{"role": "system", "content": primer}, {"role": "user", "content": context + " Generate questions with 4 options and underline the correct one."}]
        res = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=1500,  
            n=25  # Number of completions (questions with answers) to generate per iteration
        )

        # Append the generated questions and answers to the list
        for choice in res.choices:
            augmented_queries_MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary.append(choice.message.content)
            total_questions += 1
            if total_questions >= 50:
                break


# Display the augmented queries with answers
for idx, query in enumerate(augmented_queries_MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary):
    display(Markdown(f"Query {idx + 1} for MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary:\n\n{query}"))
    print("=" * 50)




Query 1 for MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary:

1. What are price multiples and how are they calculated?
   - A. Ratios of a stock’s market price to some measure of fundamental value per share
   - B. Ratios of a company’s market value to earnings before interest, taxes, depreciation, and amortization (EBITDA)
   - C. Ratios of total market value of all sources of a company’s capital to its operating cash flow
   - D. Ratios of book value to earnings per share

2. What is the intuition behind using enterprise value multiples?
   - A. Evaluating the market value of an entire enterprise relative to fundamental value per share
   - B. Evaluating the price of a share of stock in terms of per share earnings
   - C. Comparing the market value of a stock to expected future returns
   - D. Considering what a share buys in terms of per share net assets

3. How do momentum indicators differ from price and enterprise value multiples?
   - A. Momentum indicators focus on past price movements while multiples focus on future earnings
   - B. Momentum indicators evaluate an enterprise’s total value, whereas multiples focus on per share value
   - C. Multiples summarize market value relationships, while momentum indicators forecast future returns
   - D. Multiples are used for individual securities, while momentum indicators are used for broader market trends

4. What type of information can momentum indicators provide according to the text?
   - A. Historic market performance of a company
   - B. Fundamental analysis of a company’s earnings
   - C. Patterns of future returns over a certain time period
   - D. Market value of a company's total capital

**_Underlined Correct Answers:_**
1. What are price multiples and how are they calculated?
   - A. Ratios of a stock’s market price to some measure of fundamental value per share

2. What is the intuition behind using enterprise value multiples?
   - A. Evaluating the market value of an entire enterprise relative to fundamental value per share

3. How do momentum indicators differ from price and enterprise value multiples?
   - C. Multiples summarize market value relationships, while momentum indicators forecast future returns

4. What type of information can momentum indicators provide according to the text?
   - C. Patterns of future returns over a certain time period

Query 2 for MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary:

What are some common themes when using price and enterprise value multiples as valuation tools?

A) Multiples provide information solely on future patterns of returns in the short term  
B) Multiples are complex and difficult to communicate effectively with investors  
C) Multiples summarize the relationship between market value and fundamental quantities  
D) Multiples are only useful for evaluating individual securities  

**_C) Multiples summarize the relationship between market value and fundamental quantities_**

Query 3 for MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary:

What are the key features of valuation multiples as discussed in the provided text?

A) Valuation multiples provide complex and detailed insights into a company's financial health.
B) Valuation multiples are difficult to understand and apply in investment decision-making.
C) Valuation multiples offer simplicity of use and ease of communication.
D) Valuation multiples are only relevant for analyzing individual securities.

**Correct answer: C) Valuation multiples offer simplicity of use and ease of communication.**

Query 4 for MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary:

1. What are the main types of valuation tools commonly used in investment analysis?
   - A) Price multiples and revenue multiples
   - B) Enterprise value multiples and momentum indicators
   - C) Cash flow multiples and book value multiples
   - D) Price multiples and enterprise value multiples

2. How do price multiples differ from enterprise value multiples in terms of the measures they relate to fundamental value?
   - A) Price multiples relate to total market value while enterprise value multiples relate to per share value.
   - B) Price multiples relate to per share value while enterprise value multiples relate to total market value.
   - C) Price multiples relate to future expectations while enterprise value multiples relate to historical data.
   - D) Price multiples relate to cash flow while enterprise value multiples relate to revenue.

3. What is the main advantage of using multiples as valuation tools?
   - A) Complexity in calculation
   - B) Difficulty in communication
   - C) Simplicity in use and ease in communication
   - D) Limited applicability in different sectors

4. How are momentum indicators different from price and enterprise value multiples as valuation indicators?
   - A) Momentum indicators focus on a company's total capital value.
   - B) Momentum indicators predict short-term stock price movements.
   - C) Price and enterprise value multiples focus on expected future value.
   - D) Momentum indicators provide information on future patterns of returns and complement the focus of multiples.

Query 5 for MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary:

What are some common themes related to the use of price and enterprise value multiples?

A) Multiples provide a single number summary of the relationship between stock market value and fundamental quantities.
B) Multiples are complex and challenging to use for valuation purposes.
C) Multiples do not consider past performance of a company.
D) Multiples are not widely used in financial analysis.

_**A) Multiples provide a single number summary of the relationship between stock market value and fundamental quantities.**_

...

Query 46 for MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary:

1. What are price multiples and enterprise value multiples commonly used for?
   A. Analyzing market volatility
   B. Predicting interest rates
   C. Evaluating a stock's valuation
   D. Assessing political risk
**_Correct answer: C. Evaluating a stock's valuation_**

2. How do price multiples and enterprise value multiples differ?
   A. Price multiples focus on market value, while enterprise value multiples focus on total capital value
   B. Price multiples relate to earnings before interest, taxes, depreciation, and amortization (EBITDA), while enterprise value multiples relate to per share earnings
   C. Price multiples are used internationally, while enterprise value multiples are only used domestically
   D. Price multiples are complex to calculate, while enterprise value multiples are simple to use
**_Correct answer: A. Price multiples focus on market value, while enterprise value multiples focus on total capital value_**

3. What is the main appeal of using multiples as valuation tools?
   A. Simplicity in use and ease in communication
   B. Complexity in calculation for accurate results
   C. Ambiguity in interpretation for strategic advantage
   D. Lack of relevance to a company's financial performance
**_Correct answer: A. Simplicity in use and ease in communication_**

4. How do momentum indicators differ from price and enterprise value multiples?
   A. Momentum indicators focus on past values, while multiples focus on future expectations
   B. Momentum indicators are used for long-term investment, while multiples are for short-term trading
   C. Momentum indicators relate to market value, while multiples relate to financial performance
   D. Momentum indicators provide information on future returns, while multiples assess current value
**_Correct answer: A. Momentum indicators focus on past values, while multiples focus on future expectations_**

Query 47 for MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary:

1. Which valuation tool relates the total market value of all sources of a company’s capital to a measure of fundamental value for the entire company?
   A. Price multiples
   B. Valuation indicators
   C. Enterprise value multiples
   D. Momentum indicators

2. What do price multiples relate a stock’s market price to?
   A. Past values
   B. Expected values
   C. Fundamental values per share
   D. Enterprise value indicators

3. What information do momentum indicators provide regarding future patterns of returns?
   A. Historical returns
   B. Past earnings
   C. Future earnings
   D. Potential investment opportunities

4. How are price multiples summarized?
   A. In a single number relationship between market value and fundamentals
   B. By comparing sales to earnings
   C. By evaluating expected values
   D. By considering historical patterns of returns

Query 48 for MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary:

Which section of the reading discusses momentum indicators used as valuation tools?

1. Section 2 on economic context
2. Section 4 on enterprise value multiples
3. Section 5 on international considerations
4. Section 6 on momentum indicators

_**Correct:** Section 6 on momentum indicators_

Query 49 for MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary:

Which of the following is true about price and enterprise value multiples as valuation tools?

A) Price multiples compare the market value of a company's stock to its total capital value.
B) Enterprise value multiples relate a stock's market price to some measure of fundamental value per share.
C) Price multiples summarize the relationship between a company's market value and some fundamental quantity on a per-share basis.
D) Enterprise value multiples focus on evaluating the market value of an entire enterprise in isolation.

_The correct answer is C) Price multiples summarize the relationship between a company's market value and some fundamental quantity on a per-share basis._

Query 50 for MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary:

What is the focus of momentum indicators in securities selection?
1. Comparing market value of stock to fundamental quantity
2. Evaluating the market value of an entire enterprise
3. Identifying potentially rewarding investment opportunities
4. Analyzing past values of earnings per share

Correct answer: _3. Identifying potentially rewarding investment opportunities_

## Storing the 50 Q&A for Market Based Valuation Price and Enterprise Value Multiples 2018 (SetA)

In [27]:
from openai import OpenAI

# Initialize OpenAI API key
api_key = os.getenv("api_key")
client = OpenAI(api_key=api_key)

import uuid

# Assuming 'augmented_queries_MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary' is already a list of tuples (question, answer)
augmented_queries = augmented_queries_MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary  # Directly use the list

# Function to create embeddings with retry logic
def create_embeddings_with_retry(texts, model, max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            # Ensure that 'input' is a list of strings
            if not isinstance(texts, list):
                texts = [texts]
            return client.embeddings.create(input=texts, model=model)
        except Exception as e:
            print(f"Attempt {retries + 1}: Encountered an error - {e}")
            sleep(5)  # wait before retrying
            retries += 1
    raise Exception(f"Failed to create embeddings after {max_retries} retries.")

# Extract questions and answers for embedding
texts_to_embed = [qa[0] + " " + qa[1] for qa in augmented_queries]  # Combine questions and answers for embedding

# Generate embeddings for the questions and answers
embed_model = "text-embedding-3-small"
qa_embeddings = create_embeddings_with_retry(texts_to_embed, embed_model)

# Prepare data for Pinecone upsert
upsert_data = []
for idx, qa_combined in enumerate(augmented_queries):
    if not isinstance(qa_combined, str):
        print(f"Skipping item at index {idx} because it is not a string.")
        continue

    # Assuming you have a way to split qa_combined into question and answer
    parts = qa_combined.split('?', 1)
    question = parts[0].strip() + '?'
    answer = parts[1].strip() if len(parts) > 1 else ""

    # Proceed to create embeddings and upsert to Pinecone as before
    # Generate a unique ID for each query-answer pair
    qa_id = str(uuid.uuid4())
    
    # Assuming you have already generated embeddings for all QA pairs
    embed = qa_embeddings.data[idx].embedding
    
    # Prepare the upsert data with metadata
    upsert_data.append((qa_id, embed, {'question': question, 'answer': answer}))

# Perform the upsert operation to Pinecone
namespace = "SetA_MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary"  
index.upsert(vectors=upsert_data, namespace=namespace)

print(f"Upserted {len(upsert_data)} questions and answers into Pinecone index under namespace '{namespace}'.")

Upserted 50 questions and answers into Pinecone index under namespace 'SetA_MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary'.


## Generating 50 Questions for the Market Based Valuation Price and Enterprise Valyue Multiples 2018 (SetB)

In [28]:
from openai import OpenAI

# Initialize OpenAI API key
api_key = os.getenv("api_key")

# Adjust the query to indicate the need for 50 questions with answers
query = "Generate 50 questions and answers with 4 options each, with the correct option being bold, based on the Introduction Summary."

embed_model = "text-embedding-3-small"  

# Generate the embedding for the query
client = OpenAI(api_key=api_key)
res = client.embeddings.create(
    model=embed_model,
    input=query
)

# Retrieve the embedding from the response
embedding_vector = res.data[0].embedding

from IPython.display import Markdown

primer = """You are a Q&A bot, designed to provide intelligent answers based on the information provided. If the information is not available, you truthfully say, "I don't know"."""

# Initialize the list of augmented queries with answers
augmented_queries_MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary = []

# Increase the top_k parameter to retrieve more documents
query_result_MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary = index.query(vector=embedding_vector, top_k=20, namespace="MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary", include_metadata=True)

# Get list of retrieved text from the query result
contexts_MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary = [item['metadata']['text'] for item in query_result_MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary['matches']]

# Loop through each context and generate questions with answers
total_questions = 0
for context in contexts_MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary:
    # Check if we have already generated 50 questions
    if total_questions >= 50:
        break

    for _ in range(2):  # Run the loop twice for each context
        # Check if we have already generated 50 questions
        if total_questions >= 50:
            break

        # Update the message to request questions with answers
        messages = [{"role": "system", "content": primer}, {"role": "user", "content": context + " Generate questions with 4 options and underline the correct one."}]
        res = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=1500, 
            n=25  # Number of completions (questions with answers) to generate per iteration
        )

        # Append the generated questions and answers to the list
        for choice in res.choices:
            augmented_queries_MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary.append(choice.message.content)
            total_questions += 1
            if total_questions >= 50:
                break


# File path for the output file
file_path = "SetB_MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary_Questions.txt"

# Save the augmented queries with answers to a text file
with open(file_path, 'w', encoding='utf-8') as file:
    for idx, query in enumerate(augmented_queries_MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary):
        file_content = f"Query {idx + 1} for MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary:\n\n{query}\n"
        file.write(file_content)
        file.write("=" * 50 + "\n")

print(f"Output saved to {file_path}")


Output saved to SetB_MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary_Questions.txt


## Generating the 50 Q&A for the Residual Income Valuation 2018 (SetA)

In [29]:
from openai import OpenAI

# Initialize OpenAI API key
api_key = os.getenv("api_key")

# Adjust the query to indicate the need for 50 questions with answers
query = "Generate questions and answers with 4 options each, with the correct option being bold, based on the Introduction Summary. Provide 50 questions along with their answers."

embed_model = "text-embedding-3-small"  

# Generate the embedding for the query
client = OpenAI(api_key=api_key)
res = client.embeddings.create(
    model=embed_model,
    input=query
)

# Retrieve the embedding from the response
embedding_vector = res.data[0].embedding

from IPython.display import Markdown

primer = """You are a Q&A bot, designed to provide intelligent answers based on the information provided. If the information is not available, you truthfully say, "I don't know"."""

# Initialize the list of augmented queries with answers
augmented_queries_ResidualIncomeValuation2018Summary = []

# Increase the top_k parameter to retrieve more documents
query_result_ResidualIncomeValuation2018Summary = index.query(vector=embedding_vector, top_k=20, namespace="ResidualIncomeValuation2018Summary", include_metadata=True)

# Get list of retrieved text from the query result
contexts_ResidualIncomeValuation2018Summary = [item['metadata']['text'] for item in query_result_ResidualIncomeValuation2018Summary['matches']]

# Loop through each context and generate questions with answers
total_questions = 0
for context in contexts_ResidualIncomeValuation2018Summary:
    # Check if we have already generated 50 questions
    if total_questions >= 50:
        break

    for _ in range(2):  # Run the loop twice for each context
        # Check if we have already generated 50 questions
        if total_questions >= 50:
            break

        # Update the message to request questions with answers
        messages = [{"role": "system", "content": primer}, {"role": "user", "content": context + " Generate questions with 4 options and underline the correct one."}]
        res = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=1500,  
            n=25  # Number of completions (questions with answers) to generate per iteration
        )

        # Append the generated questions and answers to the list
        for choice in res.choices:
            augmented_queries_ResidualIncomeValuation2018Summary.append(choice.message.content)
            total_questions += 1
            if total_questions >= 50:
                break


# Display the augmented queries with answers
for idx, query in enumerate(augmented_queries_ResidualIncomeValuation2018Summary):
    display(Markdown(f"Query {idx + 1} for ResidualIncomeValuation2018Summary:\n\n{query}"))
    print("=" * 50)




Query 1 for ResidualIncomeValuation2018Summary:

What is the focus of residual income models in equity valuation?

A) Ignoring common shareholders' opportunity cost  
**B) Recognizing costs of all capital used in generating income**  
C) Overstating a company's net income  
D) Excluding the cost of equity capital

Query 2 for ResidualIncomeValuation2018Summary:

What historical concept does residual income date back to?

A) Principles of Economics by Alfred Marshall
B) Management Accounting Principles by Young
C) Cost Accounting Practices by Lo and Lys
D) Economic Value Added Concepts

The correct answer is _A) Principles of Economics by Alfred Marshall_.

Query 3 for ResidualIncomeValuation2018Summary:

What are some questions you can generate with 4 options to test understanding of the concept of residual income models presented in the text?

1. What is residual income?
   A) Net income minus cost of debt capital
   B) Remaining income after deducting cost of equity capital
   C) Gross income before any deductions
   D) Income calculated after excluding operating expenses

2. Why do residual income models offer advantages over traditional accounting methods?
   A) They do not consider any costs
   B) They factor in all costs of capital
   C) They ignore the cost of equity capital
   D) They solely rely on the income statement

3. Which historical figure is associated with the concept of residual income?
   A) Alfred Marshall
   B) John Maynard Keynes
   C) Adam Smith
   D) Karl Marx

4. How do residual income models contribute to estimating the intrinsic value of common stock?
   A) They focus only on current earnings
   B) They do not consider the cost of capital
   C) They discount future income streams to present value
   D) They solely rely on the book value of assets

_1. B
_2. B
_3. A
_4. C

Query 4 for ResidualIncomeValuation2018Summary:

What is the history of residual income as described in the text?

A) Alfred Marshall introduced the concept of residual income in the late 1800s.
B) General Motors utilized residual income in evaluating business segments in the 1920s.
C) Residual income has been alternatively referred to as economic profit, abnormal earnings, or economic value added.
D) All of the above.

**Correct answer: D) All of the above.**

Query 5 for ResidualIncomeValuation2018Summary:

What historical figure is credited with the development of the concept of residual income?
A) John Maynard Keynes
B) Adam Smith
C) Alfred Marshall
D) Karl Marx

**_C) Alfred Marshall_**

...

Query 46 for ResidualIncomeValuation2018Summary:

What historical figure is associated with the concept of residual income?

A) John Doe  
B) Mark Smith  
C) Alfred Marshall  
D) Emily Johnson  

**Correct Answer: C) Alfred Marshall**

Query 47 for ResidualIncomeValuation2018Summary:

What are some key points regarding residual income models according to the passage provided?

A) Traditional accounting methods adequately account for the cost of equity capital.
B) Residual income is the net income after considering the costs of a company's debt capital.
C) Residual income models recognize the costs of all capital used in generating income.
D) Residual income concept was first introduced by General Motors in the 1920s.

_**Correct Answer:** C) Residual income models recognize the costs of all capital used in generating income._

Query 48 for ResidualIncomeValuation2018Summary:

I'm sorry, but I cannot generate questions with specific options and underline the correct one. I can provide answers and information based on the text you provided. If you have any questions or need clarification, feel free to ask.

Query 49 for ResidualIncomeValuation2018Summary:

What are the key sections covered in the reading on residual income models for estimating the intrinsic value of common stock?

A) Introduction to economic profit, application in corporate performance, intrinsic value estimation  
B) Development of residual income concept, model application in valuing common stock, strengths and weaknesses  
C) Historical background, economic profit calculation, single-stage residual income model  
D) Accounting issues in residual income valuation, alternative measures, multistage residual income model  

_**Correct answer: B)**_ Development of residual income concept, model application in valuing common stock, strengths and weaknesses

Query 50 for ResidualIncomeValuation2018Summary:

1. Who developed the concept of residual income?
   - A) Robert Smith
   - B) Alfred Marshall
   - C) John Johnson
   - D) William Williams
   **_Correct answer: B) Alfred Marshall_**

2. What does residual income represent in the context of equity valuation?
   - A) Total income
   - B) Net worth
   - C) Remaining income after considering all capital costs
   - D) Total expenses
   **_Correct answer: C) Remaining income after considering all capital costs_**

3. What is one of the shortcomings of traditional accounting mentioned in the text?
   - A) Lack of income statement
   - B) Inclusion of cost of equity capital
   - C) Inclusion of residual income
   - D) Failure to account for cost of equity capital
   **_Correct answer: D) Failure to account for cost of equity capital_**

4. When was residual income first used by General Motors in evaluating business segments?
   - A) 1800s
   - B) 1920s
   - C) 1990s
   - D) 2000s
   **_Correct answer: B) 1920s_**

## Storing 50 Q&A for the Residual Income Valuation 2018 (SetA)

In [30]:
from openai import OpenAI

# Initialize OpenAI API key
api_key = os.getenv("api_key")
client = OpenAI(api_key=api_key)

import uuid

# Assuming 'augmented_queries_ResidualIncomeValuation2018Summary' is already a list of tuples (question, answer)
augmented_queries = augmented_queries_ResidualIncomeValuation2018Summary  

# Function to create embeddings with retry logic
def create_embeddings_with_retry(texts, model, max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            # Ensure that 'input' is a list of strings
            if not isinstance(texts, list):
                texts = [texts]
            return client.embeddings.create(input=texts, model=model)
        except Exception as e:
            print(f"Attempt {retries + 1}: Encountered an error - {e}")
            sleep(5)  # wait before retrying
            retries += 1
    raise Exception(f"Failed to create embeddings after {max_retries} retries.")

# Extract questions and answers for embedding
texts_to_embed = [qa[0] + " " + qa[1] for qa in augmented_queries]  # Combine questions and answers for embedding

# Generate embeddings for the questions and answers
embed_model = "text-embedding-3-small"
qa_embeddings = create_embeddings_with_retry(texts_to_embed, embed_model)

# Prepare data for Pinecone upsert
upsert_data = []
for idx, qa_combined in enumerate(augmented_queries):
    if not isinstance(qa_combined, str):
        print(f"Skipping item at index {idx} because it is not a string.")
        continue

    # Assuming you have a way to split qa_combined into question and answer
    parts = qa_combined.split('?', 1)
    question = parts[0].strip() + '?'
    answer = parts[1].strip() if len(parts) > 1 else ""

    # Proceed to create embeddings and upsert to Pinecone as before
    # Generate a unique ID for each query-answer pair
    qa_id = str(uuid.uuid4())
    
    # Assuming you have already generated embeddings for all QA pairs
    embed = qa_embeddings.data[idx].embedding
    
    # Prepare the upsert data with metadata
    upsert_data.append((qa_id, embed, {'question': question, 'answer': answer}))

# Perform the upsert operation to Pinecone
namespace = "SetA_ResidualIncomeValuation2018Summary"  
index.upsert(vectors=upsert_data, namespace=namespace)

print(f"Upserted {len(upsert_data)} questions and answers into Pinecone index under namespace '{namespace}'.")

Upserted 50 questions and answers into Pinecone index under namespace 'SetA_ResidualIncomeValuation2018Summary'.


## Generating 50 Questions for Residual Income Valuation 2018 (SetB)

In [31]:
from openai import OpenAI

# Initialize OpenAI API key
api_key = os.getenv("api_key")

# Adjust the query to indicate the need for 50 questions with answers
query = "Generate questions and answers with 4 options each, with the correct option being bold, based on the Introduction Summary. Provide 50 questions along with their answers."

embed_model = "text-embedding-3-small"  

# Generate the embedding for the query
client = OpenAI(api_key=api_key)
res = client.embeddings.create(
    model=embed_model,
    input=query
)

# Retrieve the embedding from the response
embedding_vector = res.data[0].embedding

from IPython.display import Markdown

primer = """You are a Q&A bot, designed to provide intelligent answers based on the information provided. If the information is not available, you truthfully say, "I don't know"."""

# Initialize the list of augmented queries with answers
augmented_queries_ResidualIncomeValuation2018Summary = []

# Increase the top_k parameter to retrieve more documents
query_result_ResidualIncomeValuation2018Summary = index.query(vector=embedding_vector, top_k=20, namespace="ResidualIncomeValuation2018Summary", include_metadata=True)

# Get list of retrieved text from the query result
contexts_ResidualIncomeValuation2018Summary = [item['metadata']['text'] for item in query_result_ResidualIncomeValuation2018Summary['matches']]

# Loop through each context and generate questions with answers
total_questions = 0
for context in contexts_ResidualIncomeValuation2018Summary:
    # Check if we have already generated 50 questions
    if total_questions >= 50:
        break

    for _ in range(2):  # Run the loop twice for each context
        # Check if we have already generated 50 questions
        if total_questions >= 50:
            break

        # Update the message to request questions with answers
        messages = [{"role": "system", "content": primer}, {"role": "user", "content": context + " Generate questions with 4 options and underline the correct one."}]
        res = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=1500,  
            n=25  # Number of completions (questions with answers) to generate per iteration
        )

        # Append the generated questions and answers to the list
        for choice in res.choices:
            augmented_queries_ResidualIncomeValuation2018Summary.append(choice.message.content)
            total_questions += 1
            if total_questions >= 50:
                break


# File path for the output file
file_path = "SetB_ResidualIncomeValuation2018Summary_Questions.txt"

# Save the augmented queries with answers to a text file
with open(file_path, 'w', encoding='utf-8') as file:
    for idx, query in enumerate(augmented_queries_ResidualIncomeValuation2018Summary):
        file_content = f"Query {idx + 1} for ResidualIncomeValuation2018Summary:\n\n{query}\n"
        file.write(file_content)
        file.write("=" * 50 + "\n")

print(f"Output saved to {file_path}")




Output saved to SetB_ResidualIncomeValuation2018Summary_Questions.txt


## Retrival of the Summary

In [32]:
# Initialize the list of augmented queries
augmented_queries = []

# Loop through each namespace and retrieve the corresponding query result
for namespace_name in ["FreeCashFlowValuation2018Summary", "IndustryandCompanyAnalysis2018Summary", "MarketBasedValuationPriceandEnterpriseValueMultiples2018Summary", "ResidualIncomeValuation2018Summary"]:
    # Perform the query using the embedding vector
    query_result = index.query(vector=embedding_vector, top_k=5, namespace=namespace_name, include_metadata=True)
    
    # Get list of retrieved text from the query result
    contexts = [item['metadata']['text'] for item in query_result['matches']]
    
    # Append the list of contexts to the augmented_queries list
    augmented_queries.append(contexts)

# Print the contexts from each augmented query
for idx, contexts in enumerate(augmented_queries):
    print(f"Context {idx + 1} :")
    for context in contexts:
        print(context)
    print("\n" + "=" * 50 + "\n")  


Context 1 :
Discounted cash flow (DCF) valuation views the intrinsic value of a security as the
present value of its expected future cash flows. When applied to dividends, the DCF
model is the discounted dividend approach or dividend discount model (DDM). This
reading extends DCF analysis to value a company and its equity securities by valuing
free cash flow to the firm (FCFF) and free cash flow to equity (FCFE). Whereas dividends
are the cash flows actually paid to stockholders, free cash flows are the cash
flows available for distribution to shareholders. Unlike dividends, FCFF and FCFE are not readily available data. Analysts need to
compute these quantities from available financial information, which requires a clear
understanding of free cash flows and the ability to interpret and use the information
correctly. Forecasting future free cash flows is also a rich and demanding exercise.
The analyst’s understanding of a company’s financial statements, its operations, its
financing, an